In [1]:
## inputs
# water_xyz = water = """3

# H\t0.7493682\t0.0\t0.4424329
# O\t0.0\t0.0\t-0.1653507
# H\t-0.7493682\t0.0\t0.4424329
# #     """
# geometry = water_xyz[3:]

methane = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]

geometry = methane

In [10]:
from nbed.embed import nbed_driver


n_active_atoms=2
basis = 'STO-3G'
xc_functional = 'B3LYP'# 'lda, vwn' #'B3LYP'
convergence = 1e-6
loc_method = "spade" #"spade" #"pipekmezey" "ibo" "boys"
proj_method = 'huzinaga' # 'huzinaga' #"mu_shift"
mu_level_shift=1e6
run_ccsd = True
run_fci = True
max_ram = 8_000

output = None

# q_ham, classical_energy = nbed_driver(
#     geometry ,
#     n_active_atoms,
#     basis,
#     xc_functional,
#     output,
#     convergence,
#     localization_method= loc_method,
#     projector_method = proj_method,
#     mu_level_shift = mu_level_shift,
#     run_ccsd_emb = run_ccsd,
#     max_ram_memory = max_ram,
#     run_fci_emb=run_fci)

outdic = nbed_driver(
    geometry ,
    n_active_atoms,
    basis,
    xc_functional,
    output,
    convergence,
    localization_method= loc_method,
    projector_method = proj_method,
    mu_level_shift = mu_level_shift,
    run_ccsd_emb = run_ccsd,
    max_ram_memory = max_ram,
    run_fci_emb=run_fci)


embedded HF energy MU_SHIFT: -16.514892639963357, converged: True
CCSD Energy:
	%s -39.35844359505427
embedded HF energy HUZINAGA: -16.514892561936414, converged: True
CCSD Energy:
	%s -39.35844351687314
difference between CCSD Huz and Mu calcs: 7.818113090252155e-08
difference between FCI Huz and Mu calcs: 1.6910564578975738
core constant: 0
core constant: 0
num e emb: 4
[0 1]
[2 3 4]


 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 5.56e-10
embedded HF energy MU_SHIFT: -16.089342634065826, converged: True
CCSD Energy:
	%s -39.65265921706925
embedded HF energy HUZINAGA: -16.089342579892165, converged: True
CCSD Energy:
	%s -39.65265916285197
difference between CCSD Huz and Mu calcs: 5.4217281331148115e-08
difference between FCI Huz and Mu calcs: 0.2105402360104094
core constant: 0
core constant: 0
num e emb: 4
[0 2]
[1 3 4]

In [3]:
-16.514892640009133 + 16.514892561936392

-7.807274116089502e-08

In [11]:
qubit_H_mu = outdic['mu_shift']['H']
qubit_H_huz = outdic['huzinaga']['H']

from openfermion.linalg import get_sparse_operator
H_sparse_mu = get_sparse_operator(qubit_H_mu)
H_sparse_huz = get_sparse_operator(qubit_H_huz)

In [12]:
import scipy as sp
eig_vals_emb_mu, eig_vecs_emb_mu = sp.sparse.linalg.eigsh(H_sparse_mu,
                                                   which='SA',
                                                   k=1)
del H_sparse_mu

eig_vals_emb_huz, eig_vecs_emb_huz = sp.sparse.linalg.eigsh(H_sparse_huz,
                                                   which='SA',
                                                   k=1)
del H_sparse_huz

In [13]:
import numpy as np
mu_classical_E = outdic['mu_shift']['energy_classical']
huz_classical_E = outdic['huzinaga']['energy_classical']

mu_full_quantum = eig_vals_emb_mu[0] + mu_classical_E
huz_full_quantum =eig_vals_emb_huz[0] + huz_classical_E

print(mu_full_quantum)
print(huz_full_quantum)

print(np.abs(huz_full_quantum-mu_full_quantum))

-42.706293955921666
-42.706293980245334
2.4323668412762345e-08


In [7]:
eig_vals_emb_mu[0] + mu_classical_E

-39.961698729409086

In [8]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(q_ham)

NameError: name 'q_ham' is not defined

In [ ]:
import scipy as sp
eig_vals_emb, eig_vecs_emb = sp.sparse.linalg.eigsh(H_sparse,
                                                   which='SA',
                                                   k=1)
del H_sparse

In [ ]:
E_VQE = eig_vals_emb[0] + classical_energy
E_VQE

In [ ]:
import numpy as np

N_elec_qubit_state = np.binary_repr(
                                    np.where(np.abs(eig_vecs_emb)>1e-2)[0][0]
                                    ).count('1')
N_elec_qubit_state

In [ ]:
act = [0,2,4]
env = [1,3]

[i for i in range(len(act), len(act) + len(env))]

In [ ]:
mu_CCSD = -75.86087423588484
mu_FCI =  -76.09254135408943
mu_VQE = -76.02674879647752

huz_CCSD = -75.85383455242645
huz_FCI =  -75.85383404201579
huz_VQE = -75.8538340123436

print(np.abs(mu_CCSD-huz_CCSD))
print(np.abs(mu_VQE-huz_VQE))
print(np.abs(mu_FCI-huz_FCI))

print(np.abs(mu_VQE-mu_FCI))
print(np.abs(huz_VQE-huz_FCI))